In [1]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

In [2]:
class CNN(Module):
	def __init__(self, numChannels, classes):
		# call the parent constructor
		super(CNN, self).__init__()
		# initialize first set of CONV => RELU => POOL layers
		self.conv11 = Conv2d(in_channels=numChannels, out_channels=32,
			kernel_size=(3, 3))
		self.relu11 = ReLU()
		self.conv12 = Conv2d(in_channels=32, out_channels=32,
			kernel_size=(3, 3))
		self.relu12 = ReLU()
		self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		# initialize second set of CONV => RELU => POOL layers
		self.conv21 = Conv2d(in_channels=32, out_channels=64,
			kernel_size=(5, 5))
		self.relu21 = ReLU()
		self.conv22 = Conv2d(in_channels=64, out_channels=64,
			kernel_size=(3, 3))
		self.relu22 = ReLU()
		self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		# initialize third set of CONV => RELU => POOL layers
		self.conv31 = Conv2d(in_channels=64, out_channels=128,
			kernel_size=(5, 5))
		self.relu31 = ReLU()
		self.conv32 = Conv2d(in_channels=128, out_channels=128,
			kernel_size=(3, 3))
		self.relu32 = ReLU()
		self.maxpool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		# initialize first set of FC => RELU layers
		self.fc1 = Linear(in_features=3*3*128, out_features=5*128)
		self.relu3 = ReLU()

		# initialize second set of FC => RELU layers
		self.fc1 = Linear(in_features=5*128, out_features=5*128)
		self.relu3 = ReLU()

		# initialize our softmax classifier
		self.fc2 = Linear(in_features=5*128, out_features=classes)
		self.logSoftmax = LogSoftmax(dim=1) #dim=1 je ze to robi po riadkoch

	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv11(x)
		x = self.relu11(x)
		x = self.conv12(x)
		x = self.relu12(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv21(x)
		x = self.relu21(x)
		x = self.conv22(x)
		x = self.relu22(x)
		x = self.maxpool2(x)
		# pass the output from the previous layer through the third
		# set of CONV => RELU => POOL layers
		x = self.conv31(x)
		x = self.relu31(x)
		x = self.conv32(x)
		x = self.relu32(x)
		x = self.maxpool3(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.relu3(x)
		# pass the output to our softmax classifier to get our output
		# predictions
		x = self.fc2(x)
		output = self.logSoftmax(x)
		# return the output predictions
		return output


In [3]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.lenet import LeNet
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
#from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time


In [4]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [16]:
imgs = plt.imread("D:\\skola\\ING\\LS2023\\NSIETE\\macky\\1.jpeg")
trainData = imgs
# testData = KMNIST(root="data", train=False, download=True,
# 	transform=ToTensor())
# calculate the train/validation split
print("[INFO] generating the train/validation split...")
numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = int(len(trainData) * VAL_SPLIT)
(trainData, valData) = random_split(trainData,
	[numTrainSamples, numValSamples],
	generator=torch.Generator().manual_seed(42))

[INFO] generating the train/validation split...


In [19]:
# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
model = CNN(
	numChannels=1,
	classes=len(trainData.dataset.classes)).to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] initializing the LeNet model...
[INFO] training the network...


In [25]:
model.train()

CNN(
  (conv11): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu11): ReLU()
  (conv12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu12): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv21): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu21): ReLU()
  (conv22): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu22): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv31): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (relu31): ReLU()
  (conv32): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu32): ReLU()
  (maxpool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=640, out_features=640, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=640, out_features=1, bias=True)
  (logSoftmax): LogSoftmax(dim=1)
)
